In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd

from threading import Thread
from CoolProp.CoolProp import PropsSI

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(norm(join(cwd, '..', '..', '..', 'standalone')))

from standalone.run_g_function import RunGFunctions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('MFRTRT_loads.csv', parse_dates=True, index_col='Date/Time')

In [ ]:
df.head(3)

In [ ]:
df = df.resample('2T').mean()
df = df.interpolate(method='linear')

In [ ]:
df.head(3)

In [ ]:
df.plot(y=['mdot [kg/s]'])

In [ ]:
df.plot(y=['Meas. Total Power [W]'])

In [ ]:
df.to_csv('in.csv', columns=['Meas. Total Power [W]', 'mdot [kg/s]'])

In [ ]:
df['Run Time'] = df.index.to_series().diff().dt.total_seconds().cumsum()
df['Run Time'].iloc[0] = 0
df['Run Time'].head(3)

In [ ]:
df.set_index('Run Time', drop=True, inplace=True)

In [ ]:
in_file = join(cwd, 'MFRTRT_input.json')
with open(in_file, 'r') as f:
    json_blob = f.read()
in_json = json.loads(json_blob)

In [ ]:
load_file = in_json['load-profile']['external']['path'] 
in_json['load-profile']['external']['path'] = join(cwd, load_file)

flow_file = in_json['flow-profile']['external']['path']
in_json['flow-profile']['external']['path'] = join(cwd, flow_file)

g_file = in_json['g-functions']['file']
in_json['g-functions']['file'] = join(cwd, g_file)

In [ ]:
in_json['load-aggregation']['type'] = 'dynamic'
in_json['simulation']['output-path'] = join(cwd, 'out_dynamic.csv')

with open(join(cwd, 'in_dynamic.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_json['load-aggregation']['type'] = 'static'
in_json['simulation']['output-path'] = join(cwd, 'out_static.csv')

with open(join(cwd, 'in_static.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_json['load-aggregation']['type'] = 'none'
in_json['simulation']['output-path'] = join(cwd, 'out_none.csv')

with open(join(cwd, 'in_none.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_static = os.path.join(os.getcwd(), 'in_static.json')
in_dynamic = os.path.join(os.getcwd(), 'in_dynamic.json')
in_none = os.path.join(os.getcwd(), 'in_none.json')

In [ ]:
def do_a_run(in_file):
    RunGFunctions(in_file).simulate()

In [ ]:
p1 = Thread(target=do_a_run, args=(in_static,))
p2 = Thread(target=do_a_run, args=(in_dynamic,))
p3 = Thread(target=do_a_run, args=(in_none,))

p1.start()
p2.start()
p3.start()

p1.join()
p2.join()
p3.join()

In [ ]:
df_static = pd.read_csv("out_static.csv", index_col='Simulation Time')

In [ ]:
df_dynamic = pd.read_csv("out_dynamic.csv", index_col='Simulation Time')

In [ ]:
df_none = pd.read_csv("out_none.csv", index_col='Simulation Time')

In [ ]:
df_static.tail()

In [ ]:
start_pt = 0
end_pt = 10800

in_static = df_static['GLHE Inlet Temperature [C]'].loc[start_pt : end_pt]
out_static = df_static['GLHE Outlet Temperature [C]'].loc[start_pt : end_pt]
plt.plot(in_static, label='In (Static)', linestyle='--')
plt.plot(out_static, label='Out (Static)', linestyle='--')

in_dynamic = df_dynamic['GLHE Inlet Temperature [C]'].loc[start_pt : end_pt]
out_dynamic = df_dynamic['GLHE Outlet Temperature [C]'].loc[start_pt : end_pt]
plt.plot(in_dynamic, label='In (Dynamic)', linestyle='-.')
plt.plot(out_dynamic, label='Out (Dynamic)', linestyle='-.')

in_none = df_none['GLHE Inlet Temperature [C]'].loc[start_pt : end_pt]
out_none = df_none['GLHE Outlet Temperature [C]'].loc[start_pt : end_pt]
plt.plot(in_none, label='In (None)', linestyle='-.')
plt.plot(out_none, label='Out (None)', linestyle='-.')

in_exp = df['Inlet 1 [C]'].loc[start_pt : end_pt]
out_exp = df['Outlet 1 [C]'].loc[start_pt : end_pt]
plt.plot(in_exp, label='In (Exp)', marker='x')
plt.plot(out_exp, label='Out (Exp)', marker='x')

plt.legend()

plt.show()

In [ ]:
plt.plot(df_static['Flow Fraction [-]'].loc[start_pt : end_pt], label='Static')
plt.plot(df_dynamic['Flow Fraction [-]'].loc[start_pt : end_pt], label='Dynamic')
plt.plot(df_none['Flow Fraction [-]'].loc[start_pt : end_pt], label='None')
plt.legend()
plt.show()

In [ ]:
plt.plot(df['Meas. Total Power [W]'].loc[start_pt : end_pt], label="Exp Data", marker='x')
plt.plot(df_static['GLHE Heat Transfer Rate [W]'].loc[start_pt : end_pt], label="Static")
plt.plot(df_dynamic['GLHE Heat Transfer Rate [W]'].loc[start_pt : end_pt], label="Dynamic")
plt.plot(df_none['GLHE Heat Transfer Rate [W]'].loc[start_pt : end_pt], label="None")

plt.legend()
plt.show()

In [ ]:
plt.plot(df_static['Load on GHE [W/m]'].loc[start_pt : end_pt], label='Static')
plt.plot(df_dynamic['Load on GHE [W/m]'].loc[start_pt : end_pt], label='Dynamic') 
plt.plot(df_none['Load on GHE [W/m]'].loc[start_pt : end_pt], label='None') 

plt.legend()
plt.show()

In [ ]:
df['MFT'] = df[['Inlet 1 [C]', 'Outlet 1 [C]']].mean(axis=1)

In [ ]:
plt.plot(df_static['Average Fluid Temp [C]'].loc[start_pt : end_pt], label="Static")
plt.plot(df_dynamic['Average Fluid Temp [C]'].loc[start_pt : end_pt], label="Dynamic")
plt.plot(df_none['Average Fluid Temp [C]'].loc[start_pt : end_pt], label="None")
plt.plot(df['MFT'].loc[start_pt : end_pt], label="Exp", marker='x')

# plt.ylim(16, 26)

plt.legend()
plt.show()

In [ ]:
plt.plot(df_static["Local Borehole Resistance 'Rb' [K/(W/m)]"].loc[start_pt : end_pt], label="Static")
plt.plot(df_dynamic["Local Borehole Resistance 'Rb' [K/(W/m)]"].loc[start_pt : end_pt], label="Dynamic")
plt.plot(df_none["Local Borehole Resistance 'Rb' [K/(W/m)]"].loc[start_pt : end_pt], label="None")

plt.legend()
plt.show()

In [ ]:
plt.plot(df_static["Soil Resistance 'Rs' [K/(W/m)]"].loc[start_pt : end_pt], label="Static")
plt.plot(df_dynamic["Soil Resistance 'Rs' [K/(W/m)]"].loc[start_pt : end_pt], label="Dynamic")
plt.plot(df_none["Soil Resistance 'Rs' [K/(W/m)]"].loc[start_pt : end_pt], label="None")

plt.legend()
plt.show()